# Retail Data Pipeline: End-to-End ETL Project

## Objective
To build a robust ETL (Extract, Transform, Load) pipeline that processes raw retail sales data, cleans inconsistencies, and loads it into a local SQL Server for Power BI analysis.

## Scope
1. **Extract:** Raw CSV data with inconsistencies.
2. **Transform:** Handle missing values, standardize categorical text, and parse dates.
3. **Load:** Store in SQL Server (RetailDB).

In [3]:
# Install necessary libraries (Run this once if not installed)
%pip install pandas sqlalchemy pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import Libraries
import pandas as pd
import numpy as np

# Display settings
pd.set_option('display.max_columns', None)

## Step 1: Extraction
Loading the raw dataset from the local directory.

In [5]:
# Define the file path (Using raw string 'r' to handle backslashes)
file_path = r"D:\Data Analytics Projects\Full fledge\customer-shopping-behavior-analysis-python-sql-powerbi\data\customer_shopping_behavior_raw.csv"

# Load the data
df = pd.read_csv(file_path)

# Verify load
print(f"Data Loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

Data Loaded: 3900 rows, 19 columns


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,Transaction Date
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,2023-02-20
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,2023-12-15
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2023-05-08
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,2023-03-21
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,2023-05-26


## Step 2: Initial Data Audit
Checking for missing values, data types, and duplicates.

In [6]:
# Check info and missing values
df.info()
print("\nMissing Values:\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [7]:
# Summary statistics using .describe()
df.describe(include='all')

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,Transaction Date
count,3900.000000,3900.000000,3900,3900,3900,3900.000000,3900,3900,3900,3900,3863.000000,3900,3900,3900,3900,3900.000000,3900,3900,3900
unique,NaN,NaN,2,25,4,NaN,50,4,25,4,NaN,2,6,2,2,NaN,6,7,336
top,NaN,NaN,Male,Blouse,Clothing,NaN,Montana,M,Olive,Spring,NaN,No,Free Shipping,No,No,NaN,PayPal,Every 3 Months,2023-03-22
freq,NaN,NaN,2652,171,1737,NaN,96,1755,177,999,NaN,2847,675,2223,2223,NaN,677,584,25
mean,1950.500000,44.068462,NaN,NaN,NaN,59.764359,NaN,NaN,NaN,NaN,3.750065,NaN,NaN,NaN,NaN,25.351538,NaN,NaN,NaN
std,1125.977353,15.207589,NaN,NaN,NaN,23.685392,NaN,NaN,NaN,NaN,0.716983,NaN,NaN,NaN,NaN,14.447125,NaN,NaN,NaN
min,1.000000,18.000000,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,975.750000,31.000000,NaN,NaN,NaN,39.000000,NaN,NaN,NaN,NaN,3.100000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,NaN
50%,1950.500000,44.000000,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,3.800000,NaN,NaN,NaN,NaN,25.000000,NaN,NaN,NaN
75%,2925.250000,57.000000,NaN,NaN,NaN,81.000000,NaN,NaN,NaN,NaN,4.400000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN,NaN


In [12]:
# Checking if missing data or null values are present in the dataset

df.isnull().sum()

customer_id               0
age                       0
gender                    0
item_purchased            0
category                  0
purchase_amount_usd       0
location                  0
size                      0
color                     0
season                    0
review_rating             0
subscription_status       0
shipping_type             0
discount_applied          0
promo_code_used           0
previous_purchases        0
payment_method            0
frequency_of_purchases    0
transaction_date          0
dtype: int64

### Observation:
- `Review Rating` has missing values (~37 rows).
- Column names have spaces and are inconsistent (e.g., "Purchase Amount (USD)").
- `Transaction Date` is currently an object (string), needs to be datetime.
- `Frequency of Purchases` needs standardization.

## Step 3: Transformation

### 3.1 Handle Missing Values
We will impute missing `Review Rating` using the **median** rating of each specific `Category`. This is more accurate than a global mean.

In [8]:
# Impute Review Rating
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

# Verify
print("Missing Ratings after imputation:", df['Review Rating'].isnull().sum())

Missing Ratings after imputation: 0


### 3.2 Clean Column Headers
Standardizing all column names to `snake_case` for SQL compatibility.

In [9]:
# Clean headers: lowercase, replace spaces with underscores, remove brackets
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

print(df.columns)

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_usd', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases', 'transaction_date'],
      dtype='object')


### 3.3 Standardize Categorical Data
The `frequency_of_purchases` column contains redundant labels (e.g., "Bi-Weekly" vs "Fortnightly"). We will normalize these.

In [10]:
# Check unique values before cleaning
print("Before:", df['frequency_of_purchases'].unique())

# Standardize
frequency_map = {
    'Bi-Weekly': 'Fortnightly',
    'Every 3 Months': 'Quarterly'
}
df['frequency_of_purchases'] = df['frequency_of_purchases'].replace(frequency_map)

# Check unique values after cleaning
print("After:", df['frequency_of_purchases'].unique())

Before: ['Fortnightly' 'Weekly' 'Annually' 'Quarterly' 'Bi-Weekly' 'Monthly'
 'Every 3 Months']
After: ['Fortnightly' 'Weekly' 'Annually' 'Quarterly' 'Monthly']


### 3.4 Date Parsing
Converting `transaction_date` from String to Datetime object for time-series analysis.

In [11]:
# Convert to datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Verify data type
print(df['transaction_date'].dtype)
df.head()

datetime64[ns]


,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,transaction_date
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,2023-02-20
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,2023-12-15
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2023-05-08
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,2023-03-21
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,2023-05-26


## Step 4: Ready for Load
The data is now clean. In the next phase, we will move this logic into a modular `.py` script (`etl_pipeline.py`) to automate the load into SQL Server.